In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import pandas as pd
import numpy as np
import pickle as pk
import re
import tensorflow as tf
from tqdm import tqdm

from sentence_transformers import SentenceTransformer
from nltk import sent_tokenize
from sklearn.metrics.pairwise import cosine_similarity

import sys, os
sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname('__file__'))))

In [7]:
os.chdir('/content/drive/MyDrive/NLP/NLP-CaseLaw')

In [8]:
case_fraud = pd.read_csv('./Data/case_fraud_sent.csv', index_col=0)
case_fraud.head(1)

,판례일련번호,판례내용_전처리2
0,238021,상고이유를 판단한다. 불가벌적 사후행위에 대한 판단유사수신행위의 규제에 관한 법...


# 1. 임베딩 전 전처리

In [12]:
from nltk import sent_tokenize

sentence_tokened = []

for x in tqdm(case_fraud['판례내용_전처리2']):
    sentence_tokened.append(sent_tokenize(x))

100%|██████████| 2253/2253 [00:02<00:00, 1007.92it/s]


In [13]:
sentence_tokened[0]

['  상고이유를 판단한다.',
 '불가벌적 사후행위에 대한 판단유사수신행위의 규제에 관한 법률이하 유사수신행위법이라 한다.',
 '를 위반한 행위는 그 자체가 사기행위에 해당한다거나 사기행위를 반드시 포함한다고 할 수 없고 유사수신행위법 위반죄가 형법  의 사기죄와 구성요건을 달리하는 별개의 범죄로서 서로 보호법익이 다른 이상 유사수신행위를 한 자가 출자자에게 별도의 기망행위를 하여 유사수신행위로 조달받은 자금의 전부 또는 일부를 다시 투자받는 행위는 유사수신행위법 위반죄와 다른 새로운 보호법익을 침해하는 것으로서 유사수신행위법 위반죄의 불가벌적 사후행위가 되는 것이 아니라 별죄인 사기죄를 구성한다.',
 '원심은 판시와 같은 이유로 피고인의   피해자 공소외인에 대한 판시 사기의 점에 관한 공소사실에 대하여 유사수신행위법 에서 금지하는 유사수신행위에는 기망행위가 포함되어 있지 않고 유사수신행위법 위반죄와 사기죄는 구성요건을 달리하는 별개의 범죄로서 서로 행위의 태양이나 보호법익을 달리한다는 등의 이유로 기존 범죄인 유사수신행위법 위반죄의 가벌적 평가범위 내에 흡수되는 불가벌적 사후행위에 해당하지 않는다고 판단하였다.',
 '원심판결 이유를 위 법리 및 기록에 비추어 살펴보면 이 부분 원심판단에 일부 적절하지 않은 점이 있으나 피고인의   피해자 공소외인에 대한 판시 사기죄가 유사수신행위법 위반죄의 불가벌적 사후행위에 해당하지 않는다고 본 원심의 판단에 불가벌적 사후행위에 관한 법리를 오해함으로써 판결에 영향을 미친 잘못이 없다.',
 '나머지 상고이유에 대한 판단  이 부분 상고이유는 형사소송법  에서 정한 형보다 가벼운 형이 선고된 이 사건에서 사실오인법리오해를 내세우며 실질적으로 원심의 증거 선택 및 증명력에 관한 판단 또는 이에 기초한 사실인정을 탓하는 것이거나 원심이 인정한 사실과 다른 사실관계를 전제로 법리오해를 지적하는 취지의 주장 또는 형이 너무 무거워 부당하다는 취지의 주장에 해당하여 모두 적법한 상고이유로 볼 수 없다.',
 '결론  그러므

# 2. SBERT

### 1.1 모델 & 임베딩 로드

In [14]:
with tf.device("/device:GPU:0"):
    sbert = SentenceTransformer('jhgan/ko-sbert-sts')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.44k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/538 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

### 1.2 판례 내용 임베딩

In [15]:
embeddings = []

for sentences in tqdm(sentence_tokened):
    embedding = None
    count = 0
    for sentence in sentences:
        if embedding is None:
            embedding = sbert.encode(sentence)
        else:
            embedding += sbert.encode(sentence)
        count += 1
    embeddings.append(embedding / count)

100%|██████████| 2253/2253 [14:46<00:00,  2.54it/s]


In [16]:
embeddings[0]

array([-1.41653806e-01, -1.60605758e-01,  3.68636623e-02,  6.52821422e-01,
       -2.41468832e-01, -2.99994379e-01, -2.73143146e-02,  4.37883958e-02,
       -2.11666793e-01,  9.26877037e-02, -1.31048813e-01,  2.57698238e-01,
       -2.64902294e-01,  5.04898727e-01, -6.42141998e-01,  1.69505671e-01,
        3.70818138e-01, -7.37885118e-01, -1.91684365e-01,  6.33502066e-01,
        7.31351972e-02,  6.49322271e-02,  8.07442367e-01, -8.52900818e-02,
        4.21738662e-02, -2.93746740e-01,  2.60368139e-01, -6.16776884e-01,
       -8.95257652e-01,  4.55739349e-01,  9.14467871e-01, -3.84670019e-01,
        1.79496720e-01,  9.57393497e-02,  7.21146643e-01,  2.76040673e-01,
       -4.90965694e-01,  6.63060486e-01,  1.99357942e-01,  3.12941730e-01,
        1.09051868e-01,  7.47912467e-01,  3.47941488e-01,  1.47968695e-01,
        2.15465292e-01, -3.80534410e-01, -1.58311591e-01, -5.07910550e-01,
        4.00950104e-01, -4.97981936e-01,  1.59098709e+00,  6.91897392e-01,
        2.57045865e-01,  

In [19]:
with open("./Modeling/model/sbert_emb.pickle", "wb") as fw:
    pk.dump(embeddings, fw)